In [1]:
import boto3
import sagemaker
import os

ModuleNotFoundError: No module named 'boto3'

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# location of stored sata
data_dir = 'modelling'
# S3 location
bucket = sagemaker_session.default_bucket()
# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/audio_emotion_detection'

# upload all data to S3
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)
print('Test location: {}'.format(test_location))
print('Train location: {}'.format(train_location))

Test location: s3://sagemaker-eu-central-1-668160054588/sagemaker/audio_emotion_detection/test.csv
Train location: s3://sagemaker-eu-central-1-668160054588/sagemaker/audio_emotion_detection/train.csv


## Train estimator

In [3]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='text/csv')
s3_input_test = sagemaker.s3_input(s3_data=test_location, content_type='text/csv')

output_path = 's3://{}/{}'.format(bucket, prefix)

## Load and deploy saved model

An AWS blogpost about deployment of a keras model on SageMaker (https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/) gave me a good general idea how to proceed. First, I changed the training approach to save model in a format enabling deployment on SageMaker. The blogpost described a workaround to deploy a model, by using sagemaker.tensorflow.model.TensorFlowModel and simulating its creation with an empty train.py file. As I discovered from the Sagemaker documentation, there is now a better way to deploy directly from model artifacts (https://sagemaker.readthedocs.io/en/stable/using_tf.html#deploying-directly-from-model-artifacts).

In [ ]:
!ls export/1

In [ ]:
#Tar the entire directory and upload to Amazon S3
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix=prefix)

In [ ]:
from sagemaker.tensorflow.serving import Model
model = Model(model_data = 's3://' + bucket + '/' + prefix + '/' + 'model.tar.gz',
                        role = role)

In [ ]:
%%time
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))
